# Human Activity Recognition using Inertial sensors and Neural Networks

Elia Bonetto, Filippo Rigotto. 

Deptartment of Information Engineering, University of Padova, Italy.

Human Data Analytics, a.y. 2018/2019

## Part 1 - Data preprocessing

In [71]:
from IPython.display import Image, clear_output
import os
from google.colab import drive
drive.mount('/content/drive/')
clear_output()
!ls /content/drive/My\ Drive/hda-project
os.chdir("/content/drive/My Drive/hda-project")

bitbucket-pipelines.yml  HAR.ipynb		  LICENSE    report
dataset			 HAR-Preprocessing.ipynb  README.md  requirements.txt


In [0]:
import json
import logging

import h5py
import numpy as np
import scipy as sp
import scipy.io

import pandas as pd
pd.set_option('display.precision',3)
pd.set_option('display.float_format', '{:0.3f}'.format)

from sklearn.model_selection import train_test_split

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16,10)
mpl.rcParams['axes.grid'] = True

Here we start from the [original datasets](https://www.dlr.de/kn/desktopdefault.aspx/tabid-8500/14564_read-36508/) and process data items to have the final dataset all our net models will work on.

In [0]:
datasets = [
    'dataset/ARS_DLR_DataSet.mat',
    'dataset/ARS_DLR_DataSet_V2.mat',
    'dataset/ARS_DLR_Benchmark_Data_Set.mat'
]

We reduce the labels to detect from the original 17 down to 8.

Furthermore, we don't detect transitions, so the final number of labels we work with is 7.

In [74]:
labels = ['RUNNING', 'WALKING', 'JUMPING', 
    'STNDING','SITTING', 'XLYINGX', 'FALLING',
    'WALKUPS', 'WALKDWS',
    'JUMPVRT', 'JUMPFWD', 'JUMPBCK', 
    'TRANSUP', 'TRANSDW', 'TRNSACC', 'TRNSDCC', 'TRANSIT'
]

map_encode = { label:i for i,label in enumerate(labels) }

map_encode_8 = {
    0:0,   1:1,  2:2,  3:3,  4:4,  5:5,  6:6,
    7:1,   8:1, # walking up and downstairs = walking
    9:2,  10:2, 11:2, # jumping in place, forward and backward = jumping
    12:7, 13:7, 14:7, 15:7, 16:7 # all transitions
}

map_decode_8 = {
    0: 'running',
    1: 'walking',
    2: 'jumping',
    3: 'standing',
    4: 'sitting',
    5: 'lying',
    6: 'falling',
    7: 'transition'
}

num_classes = len(map_encode)
print("Num original classes: {}".format(num_classes))

Num original classes: 17


The dataset provides the list of labels ("activities") for each segment of tracked data, along with index bounds for each item of the list. We flat this struct to a single list.

In [0]:
def flatten_labels(labels, bounds):
    """Builds a single labels array from labels and bounds."""
    start = bounds[0::2]-1 # even positions
    stop  = bounds[1::2]   # -1: numbering starts from 1

    # start is included, stop is excluded
    res = np.ones(bounds[-1], dtype=np.uint8) * map_encode['TRANSIT']
    for i, lab in enumerate(labels):
      res[start[i] : stop[i]] = lab
    return res

The dataset contains IMU measurements referred to the sensor frame, but also provides the attitude/cosine matrix to express the measurements w.r.t. the body frame.

In [0]:
def convert_body_frame(imu_data, attitude_matrix):
    """Converts sensor frames in a data item to body frames through the attitude matrix."""
    C = attitude_matrix[1:].reshape(3,3).T

    result = imu_data.copy()
    result[1:4]  = np.dot(C, imu_data[1:4].T)  # acc
    result[4:7]  = np.dot(C, imu_data[4:7].T)  # gyro
    result[7:10] = np.dot(C, imu_data[7:10].T) # mag
    return result

For each test in a dataset, we extract the relevant data, and then flat the labels to a single long list.

The rest of the processing is postponed to operate on the full dataset.

In [0]:
summacumlaude = 0
def process_single_test(dataset, key, summacumlaude):
    imu_data, attitude_matrices, activities, activities_bounds = dataset[key][0]

    # throwing away useless nested arrays
    activities = np.array([ act[0] for act in activities[0] ])
    activities_bounds = activities_bounds[0]
    
    # integrity checks on time and length
    assert([ imu_data[i][0] == attitude_matrices[i][0] for i in range(len(imu_data)) ])
    assert(len(activities_bounds) == 2*len(activities))

    # change labels to int numbers
    activities = np.array([ map_encode[a] for a in activities ])

    # get a single array of labels instead of labels + bounds
    activities_flat = flatten_labels(activities, activities_bounds)
    #print(len(activities_flat[activities_flat == 0]))
    assert(len(imu_data) == len(activities_flat))

    return imu_data, attitude_matrices, activities_flat

We use numpy arrays to store all the collected data from every test in each dataset.

In [78]:
imu_sensor = np.empty((0,10))
attitude_mat = np.empty((0,10))
activities = np.empty((0,1), dtype=np.uint8)

# loop datasets
for ds in datasets:
    dataset = scipy.io.loadmat(ds)
    keys = [ k for k in dataset if '__' not in k ]
    # loop keys=tests
    for test in keys:
        print('Loading {}:'.format(test).ljust(52,' '), end='')
        imu, mat, act = process_single_test(dataset, test, summacumlaude)
        imu_sensor = np.append(imu_sensor, imu, axis=0)
        attitude_mat = np.append(attitude_mat, mat, axis=0)
        activities = np.append(activities, act)
        
        print('{} elements'.format(imu.shape[0]).rjust(15,' '))

assert(imu_sensor.shape==attitude_mat.shape)
num_data = imu_sensor.shape[0]
clear_output()
print('Read {} records'.format(num_data))
print('IMU data shape:   {}'.format(imu_sensor.shape))
print('Attitudes shape:  {}'.format(attitude_mat.shape))
print('Activities shape: {}'.format(activities.shape))

Read 2188965 records
IMU data shape:   (2188965, 10)
Attitudes shape:  (2188965, 10)
Activities shape: (2188965,)


We save this checkpoint, even if we don't use it.

In [0]:
#with h5py.File('dataset/ARS-raw.h5','w') as h5f:
#    h5f.create_dataset('imu_sensor', data=imu_sensor)
#    h5f.create_dataset('attitudes',  data=attitude_mat)
#    h5f.create_dataset('activities', data=activities)

In [0]:
# optional reload if messing up below
#with h5py.File('dataset/ARS-raw.h5','r') as h5f:
#    imu_sensor = h5f['imu_sensor'][:]
#    attitude_mat = h5f['attitudes'][:]
#    activities = h5f['activities'][:]

Checks

In [81]:
act,cnt = np.unique(activities, return_counts=True)
cnt_sum = sum(cnt)
cnt = cnt / 100 / 60 # 100 Hz, 60 seconds
act = np.array([labels[a].lower() for a in act])

act = np.append(act,'total')
cnt = np.append(cnt, cnt_sum)

df = pd.DataFrame(np.array([act,cnt]).T, columns=['Activity','Time (min)'])
df.index = [''] * len(df) # blank index
display(df)

#TODO other if needed

,Activity,Time (min)
,running,15.049000000000001
,walking,61.53783333333333
,jumping,4.7251666666666665
,stnding,120.77183333333333
,sitting,58.87083333333333
,xlyingx,28.218166666666665
,falling,1.9836666666666667
,walkups,5.762333333333333
,walkdws,4.846500000000001
,jumpvrt,1.0595



More processing:
- reduce the number of tracked activities to 8
- removal of items labelled as transitions
- conversion of measurements to the body frame
- removal of time column (in both measurements and attitude matrixes)

In [82]:
# remap activities
print('Old labels: {}'.format(np.unique(activities)))
activities = np.array([ map_encode_8[act] for act in activities])
print('New labels: {}'.format(np.unique(activities)))

Old labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
New labels: [0 1 2 3 4 5 6 7]


In [83]:
# remove transitions
transit_label = map_encode_8[map_encode['TRANSIT']]
transit_number = sum(activities == transit_label)
print('Transit label is {}, found {} elements'.format(transit_label, transit_number))

num_data = len(imu_sensor)
print('Old num data: {}'.format(num_data))
imu_sensor = imu_sensor[activities != transit_label]
assert(imu_sensor.shape[0] == num_data-transit_number)
attitude_mat = attitude_mat[activities != transit_label]
assert(attitude_mat.shape[0] == num_data-transit_number)
activities = activities[activities != transit_label]
assert(activities.shape[0] == num_data-transit_number)

num_data = len(imu_sensor)
print('New num data: {}'.format(num_data))

num_labels = len(np.unique(activities))
print('New num labels: {}'.format(num_labels))

Transit label is 7, found 361319 elements
Old num data: 2188965
New num data: 1827646
New num labels: 7


In [0]:
# convert from sensor frame to body frame
imu_body = imu_sensor.copy()
for i, imu in enumerate(imu_sensor):
    imu_body[i] = convert_body_frame(imu, attitude_mat[i])

In [0]:
# remove time column from data
imu_sensor = imu_sensor[:,1:]
imu_body = imu_body[:,1:]
attitude_mat = attitude_mat[:,1:]

Checks

In [86]:
act,cnt = np.unique(activities, return_counts=True)
cnt_sum = sum(cnt)
cnt = cnt / 100 / 60
act = np.array([map_decode_8[a] for a in act])

act = np.append(act,'total')
cnt = np.append(cnt, cnt_sum)

df = pd.DataFrame(np.array([act,cnt]).T, columns=['Activity','Time (min)'])
df.index = [''] * len(df)
display(df)

#TODO other if needed

,Activity,Time (min)
,running,15.049000000000001
,walking,72.14666666666668
,jumping,7.5675
,standing,120.77183333333333
,sitting,58.87083333333333
,lying,28.218166666666665
,falling,1.9836666666666667
,total,1827646.0


Normalization

In [0]:
#TODO

Checks

In [0]:
#TODO

We now save the entire dataset, even if we won't use this directly. 

In [0]:
with h5py.File('dataset/ARS-full.h5','w') as h5f:
    h5f.create_dataset('imu_sens', data=imu_sensor)
    h5f.create_dataset('imu_body', data=imu_body)
    h5f.create_dataset('attitudes', data=attitude_mat)
    h5f.create_dataset('activities', data=activities)

We pre-compute and save a 80/20 train/test split of the dataset, using body-referenced data and corresponding labels. 

This is the final version we will use for training all our models.

`random_state` is the seed of the PRNG.

In [90]:
X_train, X_test, Y_train, Y_test = \
    train_test_split(imu_body, activities, test_size=0.2, random_state=1)

print("IMU shape:     " + str(imu_body.shape))
print("Labels shape:  " + str(activities.shape))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape:  " + str(X_test.shape))
print("Y_test shape:  " + str(Y_test.shape))

IMU shape:     (1827646, 9)
Labels shape:  (1827646,)
X_train shape: (1462116, 9)
Y_train shape: (1462116,)
X_test shape:  (365530, 9)
Y_test shape:  (365530,)


In [0]:
with h5py.File('dataset/ARS-train-test.h5','w') as h5f:
    h5f.create_dataset('X_train', data=X_train)
    h5f.create_dataset('X_test',  data=X_test)
    h5f.create_dataset('Y_train', data=Y_train)
    h5f.create_dataset('Y_test',  data=Y_test)

**Done**. Move to part 2 to see DL in action.